In [3]:
#import all libraries
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Input, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.optimizers import RMSprop, Adam
from keras import backend as K

In [4]:
#get and split low_res data
height = 299
width = 299
lr_batch_size = 4

lr_train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    validation_split=0.2)

lr_train_generator = lr_train_datagen.flow_from_directory(
    'D:\.Downloads\low_res\data',#'/content/drive/My Drive/malariaNN/high_res/data', C:\Users\stolk\Desktop\Workspaces\ML\malaria\high_res\data
    target_size=(height, width),
    batch_size=lr_batch_size,
    class_mode='binary',
    subset='training')
lr_validation_generator = lr_train_datagen.flow_from_directory(
    'D:\.Downloads\low_res\data',#'/content/drive/My Drive/malariaNN/high_res/data',
    target_size=(height, width),
    batch_size=lr_batch_size,
    class_mode='binary',
    subset='validation')

lr_epochs = 15
lr_steps = 22048
lr_val_steps = 5510

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\.Downloads\\low_res\\data'

In [3]:
#build inceptionv3 algorithm and compile
input_shape = None

if K.image_data_format() == 'channels_first':
    input_shape = (3, width, height)
else:
    input_shape = (width, height, 3)

base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))

out = base_model.output
out = GlobalAveragePooling2D()(out)
out = Dense(1024, activation='relu')(out)
predictions = Dense(1, activation='sigmoid')(out)

model = Model(inputs=base_model.input, outputs=predictions)

LR = 0.0001
RMS = Adam(lr=LR)
model.compile(optimizer=RMS, loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
#train to fit low_res data
history = model.fit(
    lr_train_generator,
    epochs=lr_epochs,
    steps_per_epoch=lr_steps // lr_batch_size,
    validation_data=lr_validation_generator,
    validation_steps=lr_val_steps // lr_batch_size
)

Epoch 1/15
5512/5512 [==============================] - 525s 95ms/step - loss: 0.1621 - accuracy: 0.9509 - val_loss: 0.1913 - val_accuracy: 0.9437
Epoch 2/15
5512/5512 [==============================] - 508s 92ms/step - loss: 0.1325 - accuracy: 0.9590 - val_loss: 0.1946 - val_accuracy: 0.9261
Epoch 3/15
5512/5512 [==============================] - 509s 92ms/step - loss: 0.1165 - accuracy: 0.9625 - val_loss: 0.1258 - val_accuracy: 0.9564
Epoch 4/15
5512/5512 [==============================] - 509s 92ms/step - loss: 0.1094 - accuracy: 0.9655 - val_loss: 0.1407 - val_accuracy: 0.9530
Epoch 5/15
5512/5512 [==============================] - 508s 92ms/step - loss: 0.1045 - accuracy: 0.9661 - val_loss: 0.1914 - val_accuracy: 0.9321
Epoch 6/15
5512/5512 [==============================] - 508s 92ms/step - loss: 0.0977 - accuracy: 0.9693 - val_loss: 0.1322 - val_accuracy: 0.9532
Epoch 7/15
5512/5512 [==============================] - 508s 92ms/step - loss: 0.0918 - accuracy: 0.9693 - val_loss: 0

In [12]:
model_location = 'D:\.Downloads\models\\allinone\modelweights.h5'
base_model.save_weights(model_location)

In [23]:
#get and split high_res data
hr_batch_size = 1

hr_train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    validation_split=0.2
)

hr_train_generator = hr_train_datagen.flow_from_directory(
    'D:\.Downloads\high_res\data',#'/content/drive/My Drive/malariaNN/high_res/data',
    target_size=(height, width),
    batch_size=hr_batch_size,
    class_mode='categorical',
    subset='training')
hr_validation_generator = hr_train_datagen.flow_from_directory(
    'D:\.Downloads\high_res\data',#'/content/drive/My Drive/malariaNN/high_res/data',
    target_size=(height, width),
    batch_size=hr_batch_size,
    class_mode='categorical',
    subset='validation')

hr_epochs = 10
hr_steps = 170
hr_val_steps = 40

Found 210 images belonging to 4 classes.


In [24]:
#build inceptionv3 algorithm and compile
input_shape = None

if K.image_data_format() == 'channels_first':
    input_shape = (3, width, height)
else:
    input_shape = (width, height, 3)

final_base_model = InceptionV3(weights=model_location, include_top=False, input_tensor=Input(shape=input_shape))

out = final_base_model.output
out = GlobalAveragePooling2D()(out)
out = Dense(1024, activation='relu')(out)
predictions = Dense(4, activation='softmax')(out)

final_model = Model(inputs=final_base_model.input, outputs=predictions)
#freeze algorithm and set up for high_res
#for layer in base_model.layers:
#    layer.trainable = False

LR = 0.0001
RMS = Adam(lr=LR)
final_model.compile(optimizer=RMS, loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
#fit last 2 layers with high_res data
final_history = final_model.fit(
    hr_train_generator,
    epochs=hr_epochs,
    steps_per_epoch=hr_steps // hr_batch_size,
    validation_data=hr_validation_generator,
    validation_steps=hr_val_steps // hr_batch_size
)

ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class 'tensorflow.python.keras.preprocessing.image.DirectoryIterator'>]

In [17]:
#check accuracy
accs = 0
for i in range(10):
    accuracy = final_model.evaluate(
        hr_validation_generator,
        steps=hr_val_steps // hr_batch_size,
    )
    accs += accuracy[1]
print("Avg accuracy: " + str(accs/10))

40/40 [==============================] - 3s 79ms/step - loss: 4.7543 - accuracy: 0.1500
Avg accuracy: 0.15500000417232512
